In [1]:
import pandas as pd
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats

In [2]:
#get list of all inactive players; to be used for training/testing
inactives=pd.DataFrame(players.get_inactive_players())
inactives


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4568,1627790,Ante Zizic,Ante,Zizic,False
4569,78647,Jim Zoet,Jim,Zoet,False
4570,78648,Bill Zopf,Bill,Zopf,False
4571,78650,Matt Zunic,Matt,Zunic,False


In [7]:
test_df=inactives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [ ]:
# This function creates the label variable - whether or not a player is in the HOF
def get_hof(row):
    # sleeping to respect NBA's rate limiting
    sleep(0.5)
    awards = playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    return "Hall of Fame Inductee" in awards["DESCRIPTION"].values


test_df["HOF"] = test_df.apply(get_hof, axis=1)
test_df

,id,full_name,first_name,last_name,is_active,HOF
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False


In [45]:
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    stats=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()[0]
    #remove total entry to prevent double counting
    stats=stats[stats["TEAM_ABBREVIATION"]!= "TOT"]
    #most stats can be simply summed up
    totals=pd.concat([stats.iloc[:,6:11],stats.iloc[:,12:14],stats.iloc[:,15:17],stats.iloc[:,18:]]).sum()
    #check for any 0 attempts that may cause div/0 errors
    if totals["FGA"]==0:
        #make attempts 1 to prevent dividing by 0
        totals["FGA"]=1
        #since the attempts is 0, the corresponding makes SHOULD also be 0 logically
        #but to be safe, set the makes to 0 anyways to prevent any irregularities
        totals["FGM"]=0
    #repeat for other two shooting metrics
    if totals["FTA"]==0:
        totals["FTA"]=1
        totals["FTM"]=0
    if totals["FG3A"]==0:
        totals["FG3A"]=1
        totals["FG3M"]=0
    #basic arithmetic for career shooting splits
    return pd.concat([totals,pd.Series({"FG_PCT":totals["FGM"]/totals["FGA"],"FG3_PCT":totals["FG3M"]/totals["FG3A"],"FT_PCT":totals["FTM"]/totals["FTA"]})])
playercareerstats.PlayerCareerStats(76003).get_data_frames()[0].columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [24]:
ai=inactives[inactives["id"]==947]
ai

,id,full_name,first_name,last_name,is_active
1928,947,Allen Iverson,Allen,Iverson,False


In [46]:
get_career_stats(ai)

GP           914.000000
GS           901.000000
MIN        37582.000000
FGM         8467.000000
FGA        19906.000000
FG3M        1059.000000
FG3A        3383.000000
FTM         6375.000000
FTA         8168.000000
OREB         745.000000
DREB        2649.000000
REB         3394.000000
AST         5624.000000
STL         1983.000000
BLK          164.000000
TOV         3262.000000
PF          1777.000000
PTS        24368.000000
FG_PCT         0.425349
FG3_PCT        0.313036
FT_PCT         0.780485
dtype: float64

In [56]:
test_df=pd.concat([test_df,test_df.apply(get_career_stats,axis=1)],axis=1)
test_df

,id,full_name,first_name,last_name,is_active,HOF,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,FG_PCT,FG3_PCT,FT_PCT
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False,256.0,53.0,3200.0,620.0,...,846.0,85.0,71.0,69.0,247.0,484.0,1465.0,0.501618,0.000000,0.700935
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False,505.0,0.0,11023.0,1769.0,...,4065.0,601.0,131.0,205.0,11.0,1120.0,4557.0,0.427501,0.000000,0.727857
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True,1560.0,1476.0,57446.0,15837.0,...,17440.0,5660.0,1160.0,3189.0,2527.0,4657.0,38387.0,0.559473,0.055556,0.721410
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False,586.0,336.0,15628.0,3514.0,...,1087.0,2079.0,487.0,46.0,963.0,1106.0,8553.0,0.442402,0.353996,0.905254
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False,236.0,145.0,4809.0,720.0,...,776.0,266.0,184.0,83.0,309.0,485.0,1830.0,0.417149,0.236842,0.703214
